In [5]:
import pandas as pd
import numpy as np
import torch
import os
import sys
sys.path.append("/home/docker_current/py_files/MLCLIP_exp")
sys.path.append("/home/docker_current/py_files/finetune_exp/metaclip_arch")
sys.path.append("/home/docker_current/py_files/utils")
from MLCLIP_utils import get_text_encode_model, get_image_encode_model

# device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
df_test = pd.read_csv("/home/docker_current/datasets/test.csv")
df_test['text_features'] = None
df_test['object_img'] = None

In [7]:
df_train = pd.read_csv("/home/docker_current/datasets/train.csv")

name_emb_file = "/home/docker_current/py_files/embeddings/Roberta_Large_Vit_L_14.npy"

with open(name_emb_file, 'rb') as f:
    train_emb_text = np.load(f)
    train_emb_image = np.load(f)

df_train['text_features'] = [i for i in train_emb_text]

df_train.head(2)

,id,description,object_img,text_features
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,"[-0.24118489, 0.25865138, -0.14606075, 1.22039..."
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,"[-0.12055853, 0.15714812, 0.032904923, 0.78152..."


In [8]:
# from tqdm.notebook import tqdm

# from MLCLIP import MLCLIPPredictor
# predictor = MLCLIPPredictor(device='cuda:1')

# # ГЕНЕРИРУЕМ ЭМБЕДДИНГИ ТЕСТА
# for ind_text in tqdm(range(len(df_test))):
#     text = df_test['description'][ind_text]
#     text_features = predictor.inference_text(text)
#     # text_features = torch.Tensor(text_features)
#     # with torch.no_grad():
#     #     text_features = metaclip_model.encode_image(text_features.to(device)).cpu().detach().numpy()
#     df_test['text_features'][ind_text] = text_features

# # СОХРАНЯЕМ ЭМБЕДДИНГИ ТЕСТА
# test_text_emb = np.vstack(([i for i in df_test["text_features"]]))
# name_emb_file = "test_Roberta_Large_Vit_L_14.npy"
# with open(name_emb_file, 'wb') as f:
#     np.save(f, test_text_emb)

In [9]:
# ЗАГРУЖАЕМ ЭМБЕДДИНГИ ТЕСТА

name_emb_file = "test_Roberta_Large_Vit_L_14.npy"
with open(name_emb_file, 'rb') as f:
    test_emb_text = np.load(f)

df_test['text_features'] = [i for i in test_emb_text]
df_test.head(2)

,id,description,text_features,object_img
0,486,Фотография. Елизавета Алексеевна Юманова. ПКМ...,"[-0.13635892, 0.54791373, -0.15054585, 0.17538...",None
1,813,Фотография. Заседание комитета комсомола мотор...,"[0.026515692, 0.29201433, 0.17648302, -0.46670...",None


In [41]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

X = train_emb_text
y = df_train['object_img'].values.reshape(-1,1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.0001, random_state=42)

X.shape, y.shape

((2098, 768), (2098, 1))

In [45]:
boost = CatBoostRegressor(iterations = 1500,
                            task_type="GPU",
                            devices='0:1',
                            )

boost.fit(X_train, y_train)

# boost.fit(X_train, y_train,
#             eval_set=(X_val, y_val))

Learning rate set to 0.034939
0:	learn: 851.9589433	total: 20.3ms	remaining: 30.5s
1:	learn: 834.5979029	total: 40.2ms	remaining: 30.1s
2:	learn: 817.0341003	total: 59.7ms	remaining: 29.8s
3:	learn: 800.2460280	total: 79.4ms	remaining: 29.7s
4:	learn: 783.2399847	total: 99.6ms	remaining: 29.8s
5:	learn: 766.9201596	total: 120ms	remaining: 29.8s
6:	learn: 751.1590180	total: 140ms	remaining: 30s
7:	learn: 736.3024818	total: 161ms	remaining: 30s
8:	learn: 721.7557145	total: 181ms	remaining: 30.1s
9:	learn: 707.9690646	total: 202ms	remaining: 30s
10:	learn: 694.7249597	total: 221ms	remaining: 30s
11:	learn: 681.9618366	total: 242ms	remaining: 30s
12:	learn: 668.8211481	total: 264ms	remaining: 30.2s
13:	learn: 657.8352499	total: 281ms	remaining: 29.9s
14:	learn: 647.1021527	total: 309ms	remaining: 30.6s
15:	learn: 635.6158373	total: 334ms	remaining: 31s
16:	learn: 625.2704136	total: 354ms	remaining: 30.8s
17:	learn: 613.5686555	total: 371ms	remaining: 30.5s
18:	learn: 603.5314544	total: 388

In [46]:
X_test = test_emb_text
y_predict = boost.predict(X_test)

In [47]:
submit = pd.read_csv('/home/docker_current/py_files/sample_solution.csv')
submit['object_img'] = y_predict
submit['object_img'] = submit['object_img'].astype(np.int64)
submit.to_csv('./submits/dummy_4_catboost_full_data.csv', index=False)